In [1]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
import numpy as np
import cdd

v = np.array([[1.0, 1.0,0.0,1.0],[1.0, 0.0,2.0,1.0]])
np.linalg.matrix_rank(v)

2

In [2]:
v_min_size = min(v.shape)
dim = v.shape[1]
pca = PCA(n_components=v_min_size)
pca.fit(v)
C = pca.components_
mean = pca.mean_
low_dim_pts = pca.transform(v)
print (v)
print (C)
print (mean)
print (low_dim_pts)
print (np.matmul(C, C.T))

[[1. 1. 0. 1.]
 [1. 0. 2. 1.]]
[[-0.          0.4472136  -0.89442719 -0.        ]
 [-0.4472136   0.8         0.4         0.        ]]
[1.  0.5 1.  1. ]
[[ 1.11803399e+00  5.55111512e-17]
 [-1.11803399e+00 -5.55111512e-17]]
[[1.00000000e+00 4.08926401e-17]
 [4.08926401e-17 1.00000000e+00]]


In [3]:
print (np.matmul(C.T,low_dim_pts.T))

[[-2.48253415e-17  2.48253415e-17]
 [ 5.00000000e-01 -5.00000000e-01]
 [-1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00]]


In [4]:
explain_ratio = np.cumsum(pca.explained_variance_ratio_)

first_dims = np.argmax(explain_ratio >= 1.0-0.00001)

low_dim_pts_reduct = low_dim_pts[:,0:first_dims+1]

# decide how many to remove...
print (low_dim_pts_reduct)

[[ 1.11803399]
 [-1.11803399]]


In [5]:
if low_dim_pts_reduct.shape[1] == 1:
    lb = np.min(low_dim_pts_reduct)
    ub = np.max(low_dim_pts_reduct)
    # x <= ub  -x >= -lb
    A = np.array([[1.0],[-1.0]])
    b = np.array([ub, -lb])
else:
    # convex hull will get you Ax + b <= 0
    hull = ConvexHull(points=low_dim_pts_reduct)
    A = hull.equations[:,:-1]
    b = -hull.equations[:,-1]
print (A,b)

[[ 1.]
 [-1.]] [1.11803399 1.11803399]


In [6]:
n_rows = A.shape[0]
n_cols = A.shape[1]

num_var_to_add = v_min_size - (first_dims+1)

zero4 = np.zeros((n_rows,num_var_to_add))

rows_to_append = []
for idx in range(num_var_to_add):
    l = [0.0]*num_var_to_add
    l[idx] = 1.0
    rows_to_append.append(np.append(np.zeros((1,n_cols)), l))
    l[idx] = -1.0
    rows_to_append.append(np.append(np.zeros((1,n_cols)), l))

trueA = np.vstack([np.hstack([A, zero4])] + rows_to_append)
trueB = np.append(b, [0.0, 0.0]*num_var_to_add)
print (trueA)
print (trueB)

[[ 1.  0.]
 [-1.  0.]
 [ 0.  1.]
 [ 0. -1.]]
[1.11803399 1.11803399 0.         0.        ]


In [8]:
print (mean)
print (trueA.shape)
print (trueA)
print (C)
print (C.shape)
print (trueB)

[1.  0.5 1.  1. ]
(4, 2)
[[ 1.  0.]
 [-1.  0.]
 [ 0.  1.]
 [ 0. -1.]]
[[-0.          0.4472136  -0.89442719 -0.        ]
 [-0.4472136   0.8         0.4         0.        ]]
(2, 4)
[1.11803399 1.11803399 0.         0.        ]


In [7]:
finalA = np.matmul(trueA, C)
finalA

array([[ 0.        ,  0.4472136 , -0.89442719,  0.        ],
       [ 0.        , -0.4472136 ,  0.89442719,  0.        ],
       [-0.4472136 ,  0.8       ,  0.4       ,  0.        ],
       [ 0.4472136 , -0.8       , -0.4       ,  0.        ]])

In [10]:
np.matmul(finalA, mean)

array([-0.67082039,  0.67082039,  0.3527864 , -0.3527864 ])

In [9]:
np.matmul(finalA, mean)+trueB

array([ 0.4472136 ,  1.78885438,  0.3527864 , -0.3527864 ])

In [9]:
0.89442719/0.4472136

1.9999999776393207

In [10]:
1.78885438/0.4472136

3.9999999552786414

In [12]:
0.3527864/0.4472136

0.7888543639996637